In [10]:
import importlib
import engine
import nn
importlib.reload(engine)
importlib.reload(nn)
import kagglehub
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# Download latest version
# path = kagglehub.dataset_download("manishkc06/startup-success-prediction")


# path += '/startup data.csv'

# with open(path) as f:
#     df = pd.read_csv(f)
    
# df = df.dropna()

# y = df.pop('status')

In [11]:
X, y = load_digits(return_X_y=True)

In [12]:
t = MinMaxScaler()
t.fit(X)
X_train = t.transform(X)

X_train

array([[0.    , 0.    , 0.3125, ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.625 , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 1.    , 0.5625, 0.    ],
       ...,
       [0.    , 0.    , 0.0625, ..., 0.375 , 0.    , 0.    ],
       [0.    , 0.    , 0.125 , ..., 0.75  , 0.    , 0.    ],
       [0.    , 0.    , 0.625 , ..., 0.75  , 0.0625, 0.    ]])

In [13]:
encoder = nn.MLP(64, [32, 16])

print(encoder.layers)

decoder = nn.MLP(16, [32, 64]) 

[Layer of [ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64), ReLUNeuron(64)], Layer of [{self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32), {self.activate}Neuron(32)]]


In [14]:
print(encoder.summary())

TypeError: 'list' object is not callable

In [ ]:


def loss(batch_size):
    epsilon = 1e-8
    ri = np.random.permutation(X.shape[0])[:batch_size]
    Xb = X[ri]
    ri = np.random.permutation(y.shape[0])[:batch_size]
    yb = y[ri]
    
    inputs = [list(map(engine.Value, xrow)) for xrow in Xb]
    
    print("is me", inputs)
    
    scores = [n(input) for input in inputs]
    
    
    scores = [i.sigmoid() for i in scores]
    
    combination = zip(scores, yb)
# 
    combination = [i for i in combination if not np.isnan(i[0].data)]
    

    Binary_Cross_Entropy = [(truth * ((y + epsilon).log()) + (1 - truth) * ((1 - y + epsilon).log())) for y, truth in combination]
    
    loss = sum(Binary_Cross_Entropy) * -(1/len(scores))
    
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in n.parameters()))
    
    
    total_loss = loss + reg_loss
    
    accuracy = [(yi > 0) == (scorei.data > 0) for scorei, yi in combination]
    
    return total_loss, sum(accuracy) / len(accuracy)

print(loss(15))
        

In [ ]:
# optimization
for k in range(100):
    
    # forward
    total_loss, acc = loss(100)
    
    # backward
    n.zero_grad()
    total_loss.backward()
    
    # update (sgd)
    learning_rate = 1.0 - 0.9*k/100
    for p in n.parameters():
        p.data -= learning_rate * p.grad
    
    if k % 1 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")


In [17]:
import math
import numpy as np


class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op  # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)
                          ), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    # fix dead neuron problem
    def leaky_relu(self):
        out = Value(self.data * 0.01 if self.data <=
                    0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += out.data * \
                out.grad if out.data > 0 else (out.data * 0.01) * out.grad
            # self.grad += (out.data * 0.01) * out.grad
        out._backward = _backward

        return out

    def log(self):

        out = Value(math.log(self.data), (self, ), 'log')

        def _backward():
            self.grad += (1 / self.data) * out.grad
        out._backward = _backward

        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out

    def sigmoid(self):
        x = self.data
        t = 1 / (1 + (math.exp(-x)))

        out = Value(t, (self, ), 'sigmoid')

        def _backward():
            self.grad += (out.data * (1 - out.data)) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __ge__(self, other):
        return self.data >= other.data

    def __le__(self, other):
        return self.data <= other.data

    def __gt__(self, other):
        return self.data > other.data

    def __lt__(self, other):
        return self.data < other.data

    def __neg__(self):  # -self
        return self * -1

    def __radd__(self, other):  # other + self
        return self + other

    def __sub__(self, other):  # self - other
        return self + (-other)

    def __rsub__(self, other):  # other - self
        return other + (-self)

    def __rmul__(self, other):  # other * self
        return self * other

    def __truediv__(self, other):  # self / other
        return self * other**-1

    def __rtruediv__(self, other):  # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"


In [102]:
import random
from graphviz import Digraph
import math
import importlib


class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def zero_weights(self):
        for w in self.weights():
            w.grad = 0

    def parameters(self):
        return []

    def weights(self):
        return []

    def layers(self):
        return []

    def summary(self):
        return f"{len(self.layers())} layers, {len(self.parameters())} parameters"


class Neuron(Module):

    # I want to introduce weight sharing, which means I need to be able to
    # initialise a neuron with pre defined weights, but leave the bias?

    def __init__(self, nin, nonlin=True, **kwargs):
        self.tied = kwargs.get('tied', None)
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)] if self.tied is None else list(reversed(self.tied))
        self.b = Value(random.uniform(-1, 1))
        self.nonlin = nonlin
        self.activate = kwargs.get('activate', None)

    def __call__(self, x):
        if isinstance(x, Value):
            length = 0
            if isinstance(self.w[0], Value):
                return (self.w[0] * x) + self.b
            else:
                return (self.w[0][0] * x) + self.b
        else:
            length = len(x) - 1
        self.w = self.w[length] if self.tied is not None else self.w
        print("this is inside soon son", self.w ,"division                          ", x)
        act = sum((wi*xi for wi, xi in zip(self.w, x)),
                  self.b) if length > 0 else (self.w[0] * x) + self.b
        if self.activate and self.nonlin == False:
            return self.activate(act)
        else:
            return act.leaky_relu() if self.nonlin else act

    def parameters(self):
        return self.w + [self.b]

    def weights(self):
        # return self.w without the brackets
        return [self.w]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else '{self.activate}'}Neuron({len(self.w)})"


class Layer(Module):

    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

    def weights(self):
        # 
        # [[p for p in n.weights()] for n in self.neurons]
        return [p for n in self.neurons for p in n.weights()]
        

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"


class MLP(Module):

    def __init__(self, nin, nouts, **kwargs):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=i != len(nouts)-1, **kwargs)
                       for i in range(len(nouts))]

    def __call__(self, x):
        # print(x, "whats the value when?")
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def weights(self):
        # [[p for p in layer.weights()] for layer in self.layers]
        # 
        return [p for layer in self.layers for p in layer.weights()]
        

    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"


# class Encoder(Module):
#     def __init__(self, )


# class Decoder(Module):
    # def __init__(self, )

class AutoEncoder(Module):
    # i want this to be used so that you give it input and then you will get the
    # exact output that you used, so it will undo all the activation functions
    # but how to do?

    # self.weights() is the whole list of weights for the whole neural network
    # but i only want to pass it the weights for that layer i believe. how do
    # i do that?

    def __init__(self, in_embeds, n_hidden_layers, compressed, act_func=None, tied=False):
        # determining the number of hidden layers, which should be reduced the further you go
        n_hidden_layers = [math.ceil(in_embeds / i)
                           for i in range(2, n_hidden_layers + 2)]
        # create the layers that are not the input layer which
        # equal to hidden_layers plus the final compressed latent representation
        # layer
        self.encoder = MLP(in_embeds, (n_hidden_layers + [compressed]))
        # for the decoder do it backwards, because you're mean to build it in reverse
        # the activation function of the final layer should be a sigmoid
        # so pass it in which will go up the chain to the neuron class
        # and call it on the final layer outputs
        self.act_func = act_func
        if tied == True:
            self.decoder = MLP(
                compressed, (list(reversed(n_hidden_layers)) + [in_embeds]), tied=self.weights(), activate=act_func)
        if tied == False:
            self.decoder = MLP(
                compressed, (list(reversed(n_hidden_layers)) + [in_embeds]), activate=act_func)

    def __call__(self, x):
    
        # ze architecture vurks like you give it unt input,
        # it passes it into ze encoder, zen it compresses
        # zen it passes it into ze decoder which has activation function
        # in ze final layer then it produces output, (i was being german for the last 5 minutes yesterday, yesterday being 26/11/25)

        compressed = self.encoder(x)
        out = self.decoder(compressed)
        return out

    def parameters(self):
        return self.encoder.parameters() + self.decoder.parameters()

    def weights(self):
        return self.encoder.weights()

    def layers(self):
        return self.encoder.layers + self.decoder.layers

    def pretty(self):
        if self.act_func != None:
            hey = str(self.act_func)
            return hey.split()[1][6:]
        else:
            return "no function"

    def __repr__(self):
        return f"encoder has {self.summary()}, decoder has {self.summary()} activated with {self.pretty()}"


hey = [1.0, 2.0, 3.0, 4.0]

yb = [1.0, 0.0, 0.0, 1.0]

auto = AutoEncoder(16, 2, 4, Value.sigmoid, True)


# print(auto.weights()[0])


# is it done? the autoencoder implementation?


def trace(root):
    nodes, edges = set(), set()

    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges


def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    # , node_attr={'rankdir': 'TB'})
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir})

    for n in nodes:
        dot.node(name=str(id(n)), label="{ data %.4f | grad %.4f }" % (
            n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))

    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot


temp = auto(hey)

# print(auto.summary())

they = sum(temp)

they.backward()

draw_dot(they)


# print(np.array(auto.weights(), ndmin=2).T)

# draw_dot(they)

# mean = sum((yg - yp)**2 for yg, yp in zip(yb, temp))

# mean.backward()

# draw_dot(mean)


this is inside soon son [Value(data=0.5123421692580274, grad=0), Value(data=0.07539214512416503, grad=0), Value(data=0.2837471890416672, grad=0), Value(data=-0.9590346383117447, grad=0), Value(data=-0.03348280970528439, grad=0), Value(data=-0.43409888343310565, grad=0), Value(data=-0.4873109238044484, grad=0), Value(data=0.8435244273258449, grad=0), Value(data=0.21104587305267297, grad=0), Value(data=0.19194264479093137, grad=0), Value(data=-0.34242125145329116, grad=0), Value(data=-0.8054059243923815, grad=0), Value(data=0.8461254096966919, grad=0), Value(data=-0.8454174667019476, grad=0), Value(data=-0.458384123751425, grad=0), Value(data=-0.1663689808582276, grad=0)] division                           [1.0, 2.0, 3.0, 4.0]
this is inside soon son [Value(data=-0.5134356851524642, grad=0), Value(data=-0.9426992324304144, grad=0), Value(data=0.09295723641375031, grad=0), Value(data=0.11141958983530964, grad=0), Value(data=0.32284669023601187, grad=0), Value(data=0.7091313479418975, grad

KeyboardInterrupt: 